## Imports

In [ ]:
# Install dependencies
!pip install plotly
!pip install catboost
!pip install skforecast

In [ ]:
import pandas as pd
import plotly.express as px

from joblib import dump, load
from catboost import CatBoostRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

### Predict Function

In [ ]:
def predict_test_forecaster(data, cat, exog=[]):
    """
    
    Iterate over each day in the data and predict the next 96 values (1 Day)
    
    """
    all_predictions = pd.DataFrame(columns=["preds"])
    print(len(data))

    for i in range(0, len(data), 96):
        if i + seq_len + 97 < len(data): # if there is more data
            step_prediction = pd.DataFrame(columns=["preds"])
            
            # if there are exog variables
            if len(exog) > 0: 
                step_prediction["preds"] = cat.predict(
                    steps=96,
                    last_window=data.iloc[
                        i : i + seq_len, data.columns.get_loc("S_TOT")
                    ],
                    exog=exog,
                )
            # if there are no exog variables
            else:
                step_prediction["preds"] = cat.predict(
                    steps=96,
                    last_window=data.iloc[
                        i : i + seq_len, data.columns.get_loc("S_TOT")
                    ],
                )
            # apply datetime as index for merging the step prediction 
            # to to the data df (to have the relation S_TOT, Prediction)
            step_prediction["datetime"] = data.iloc[
                i + seq_len + 1 : i + 97 + seq_len
            ].index
            all_predictions = pd.concat([all_predictions, step_prediction])

    all_predictions = all_predictions.set_index("datetime")
    data["preds"] = all_predictions["preds"]
    data.dropna(inplace=True)
    data["preds"] = data["preds"].astype(float)
    return data

# 1. Train only on S_TOT

### 1.1 Read in Data

In [ ]:
# Settings: --impute --compensate-outliers, no scaling applied
df = pd.read_pickle("../data/not_scaled/load.pkl")
df

### 1.1.1 Optional: Add Additional Features (results get worse)

In [ ]:
# if used, add varaibles as exogenous during training (see train with autoencoder for reference)
df["time"] = df.index.strftime("%H")
df["month"] = df.index.month
df["min"] = df.index.strftime("%M")
df["hour_min"] = df["time"] + df["min"]
df["weekday"] = df.index.weekday
df["hour_min"] = df["hour_min"].astype(int)

### 1.2 Select features for Training

In [ ]:
# df2 = df[["S_TOT", "hour_min" ,"date_is_holiday", "weekday", "date_season"]]
data = df[["S_TOT"]]

### 1.3 Train Test Split

In [ ]:
data_train = data.loc["2018-05-01 00:00:00+00:00":"2019-12-31 23:45:00+00:00 "]
data_test = data.loc["2019-12-31 23:45:00+00:00":]
print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})"
)

### 1.4 Train Model

In [ ]:
# to use GPU instead of CPU use task_type="GPU"
seq_len = 2688
forecaster = ForecasterAutoreg(
    regressor=CatBoostRegressor(
        iterations=1000, task_type="CPU", grow_policy="Lossguide", has_time=True
    ),
    lags=seq_len,
)

forecaster.fit(y=data_train["S_TOT"])

### 1.5 Make Predictions

In [ ]:
# this step takes depending on the model size, data shape and window size up to 10 mins
# since it's not possible to use a prediction intervall, every step has to be done separately
prediction_df = predict_test_forecaster(data_test.copy(), forecaster)

In [ ]:
prediction_df

### 1.6 Evaluate Results

In [ ]:
print(f"MSE: {mean_squared_error(prediction_df['S_TOT'], prediction_df['preds'], squared=False)}")
print(f"RMSE: {mean_squared_error(prediction_df['S_TOT'], prediction_df['preds'])}")
print(f"R2: {r2_score(prediction_df['S_TOT'], prediction_df['preds'])}")
print(f"MAPE: {mean_absolute_percentage_error(prediction_df['S_TOT'], prediction_df['preds'])}")

### 1.7 Plot Data

In [ ]:
fig = px.line(prediction_df, x=prediction_df.index, y=["S_TOT", "preds"])
fig.show()

### 1.8 Check Feature Importance

In [ ]:
pd.set_option("display.max_rows", None)
forecaster.get_feature_importance().sort_values(by=["importance"], ascending=False)

# The most important feature by far is the first value in the window size 
# (the last 15 min value)

### 1.9 Save Model

In [ ]:
dump(forecaster, filename=f"gboost_model_{seq_len}.py")

# 2. Train with Autoencoder Features

### 2.1 Load in Data

In [ ]:
# Settings: --impute --compensate-outliers, no scaling applied
df = pd.read_pickle("../data/preprocessed_ae/load.pkl")

### 2.2 Select Features for Training

In [ ]:
df["S_TOT"] = df["TARGET"].shift(-1)
df.dropna(inplace=True)
data = df.drop("TARGET", axis=1)

### 2.3 Train Test Split

In [ ]:
data_train = data.loc["2018-05-01 00:00:00+00:00":"2019-12-31 23:45:00+00:00 "]
data_test = data.loc["2019-12-31 23:45:00+00:00":]

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})"
)
data_test_exog = data_test.drop("S_TOT", axis=1)
data_train_exog = data_train.drop("S_TOT", axis=1)

### 2.4 Train Model

In [ ]:
# to use GPU instead of CPU use task_type="GPU"
seq_len = 2688
forecaster = ForecasterAutoreg(
    regressor=CatBoostRegressor(
        iterations=1000, task_type="CPU", grow_policy="Lossguide", has_time=True, 
    ),
    lags=seq_len,
)
forecaster.fit(y=data_train["S_TOT"], exog=data_train_exog)

### 2.5 Make Predictions

In [ ]:
prediction_df = predict_test_forecaster(data_test.copy(), forecaster, data_test_exog)

In [ ]:
prediction_df

### 2.6 Evaluate Results

In [ ]:
print(f"MSE: {mean_squared_error(prediction_df['S_TOT'], prediction_df['preds'], squared=False)}")
print(f"RMSE: {mean_squared_error(prediction_df['S_TOT'], prediction_df['preds'])}")
print(f"R2: {r2_score(prediction_df['S_TOT'], prediction_df['preds'])}")
print(f"MAPE: {mean_absolute_percentage_error(prediction_df['S_TOT'], prediction_df['preds'])}")

### 2.7 Plot Results

In [ ]:
fig = px.line(prediction_df, x=prediction_df.index, y=["S_TOT", "preds"])
fig.show()

### 2.8 Check Feature Importance

In [ ]:
pd.set_option("display.max_rows", None)
forecaster.get_feature_importance().sort_values(by=["importance"], ascending=False)


# The most important feature by far is the first value in the window size 
# (the last 15 min value)


# The features from the autoencoder are not that relevant, which matches the metric results as they are not that different

### 2.9 Save Model

In [ ]:
dump(forecaster, filename=f"gboost_model_{seq_len}_ae.py")

# 3. Grid Search

### 3.1 Load data

In [ ]:
df = pd.read_pickle("../data/not_scaled/load.pkl")

data = df[["S_TOT"]]

### 3.2 Train Grid Search

In [ ]:
# to use GPU instead of CPU use task_type="GPU"
lags_grid = [96, 192, 672, 1344, 2016, 2688]
cat = ForecasterAutoreg(
    regressor=CatBoostRegressor(
        iterations=20000,
        task_type="CPU",
        verbose=False,
        grow_policy="Lossguide",
        has_time=True,
    ),
    lags=seq_len,
)
# Regressor hyperparameters
param_grid = {
    "iterations": [500, 1000, 2000, 5000, 7000, 10000],
    "depth": [16, 32, 64],
    "max_leaves": [64, 128],
    "learning_rate": [0.03, 0.1, 0.3],
    "reg_lambda": [2, 3, 6],
}
results_grid = grid_search_forecaster(
    forecaster=cat,
    y=data.loc["2018-05-01 00:00:00+00:00":"2019-12-31 23:45:00+00:00", "S_TOT"],
    param_grid=param_grid,
    lags_grid=lags_grid,
    steps=96,
    refit=False,
    metric="mean_squared_error",
    initial_train_size=len(
        data.loc["2018-05-01 00:00:00+00:00":"2019-05-01 23:45:00+00:00"]
    ),
    fixed_train_size=False,
    return_best=True,
    verbose=False,
    exog=None,  # can be changed to use multiple Features
)

# Notice, that initial_train_size limits the training size of the y data until 2019-05-01 23:45:00+00:00
# so 2019-05-01 23:45:00+00:00 - 2019-12-31 23:45:00+00:00 is used for validation


# the best results are from the model trained with:
# depth=64
# max_leaves=128
# learning_rate=default
# reg_lambda = default
# lag=2688
# iterations = 2000
# Autoencoder features as exogenous variables

# Metrics on testing dataset:
# MSE around 4.6-4.8 mio
# MAPE around 9.6 to 9.8 %
# R2 around 0.72-0.74



In [ ]:
results_grid

### 3.3 Make Predictions

In [ ]:
prediction_df = predict_test_forecaster(data_test.copy(), cat)

### 3.4 Evaluate Results

In [ ]:
print(f"MSE: {mean_squared_error(prediction_df['S_TOT'], prediction_df['preds'], squared=False)}")
print(f"RMSE: {mean_squared_error(prediction_df['S_TOT'], prediction_df['preds'])}")
print(f"R2: {r2_score(prediction_df['S_TOT'], prediction_df['preds'])}")
print(f"MAPE: {mean_absolute_percentage_error(prediction_df['S_TOT'], prediction_df['preds'])}")

### 3.5 Plot Results

In [ ]:
fig = px.line(prediction_df, x=prediction_df.index, y=["S_TOT", "preds"])
fig.show()

### 3.6 Save Model

In [ ]:
dump(cat, filename="gboost_model_grid.py")

# How to use a Pretrained Model

### Only S_TOT

In [ ]:
forecaster = load("./boosting_models/gboost_model_2688.py")
# Settings: --impute --compensate-outliers, no scaling applied
df = pd.read_pickle("../data/not_scaled/load.pkl")
seq_len = 2688
data = df[["S_TOT"]]

data_train = data.loc["2018-05-01 00:00:00+00:00":"2019-12-31 23:45:00+00:00 "]
data_test = data.loc["2019-12-31 23:45:00+00:00":]
data_test["date"] = data_test.index
print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})"
)

prediction_df = predict_test_forecaster(data_test.copy(), forecaster)

fig = px.line(prediction_df, x=prediction_df.index, y=["S_TOT", "preds"])
fig.show()

### Autoencoder Features

In [ ]:
forecaster = load("./boosting_models/gboost_model_2688_ae.py")
# Settings: --impute --compensate-outliers, no scaling applied
df = pd.read_pickle("../data/preprocessed_ae/load.pkl")

df["S_TOT"] = df["TARGET"].shift(-1)
df.dropna(inplace=True)
data = df.drop("TARGET", axis=1)

data_train = data.loc["2018-05-01 00:00:00+00:00":"2019-12-31 23:45:00+00:00 "]
data_test = data.loc["2019-12-31 23:45:00+00:00":]

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}  (n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}  (n={len(data_test)})"
)
data_test_exog = data_test.drop("S_TOT", axis=1)

prediction_df = predict_test_forecaster(data_test.copy(), forecaster, data_test_exog)

fig = px.line(prediction_df, x=prediction_df.index, y=["S_TOT", "preds"])
fig.show()